Primero importamos todas las librerías a utilizar y creamos un objeto Trainset

In [1]:
from trainset import Trainset
import dataset_manager
from fileinput import close
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from datetime import datetime


myTrainset = Trainset()

2022-10-04 16:40:53.204230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-04 16:40:53.204259: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Seteamos los valores para el entrenamiento del modelo

In [2]:
myTrainset.fecha_inicio_dataset= "2013-04-02"
myTrainset.fecha_fin_dataset= "2015-06-18"
myTrainset.dias_hacia_atras = 4
myTrainset.porcentaje_entrenamiento = 0.7
myTrainset.porcentaje_validacion = 0.15
myTrainset.porcentaje_test = 0.15

Obtenemos el dataset junto con los conjuntos de entrenamiento, validación y testeo ya divididos.

In [3]:
training_days, validation_days, test_days = dataset_manager.getDataset(myTrainset)

/home/piche/Facultad/InteligenciaComputacional/nasdaq-index-rn/src/dataset_manager.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return filtered_dataset.append(days_before_first_day)


Separamos los valores de cierre, seno del día de la semana y coseno del día de la semana en 3 listas. Tambien creamos listas para setear las inputs y outputs a utilizar en el entrenamiento.
Seteamos las inputs del output y entrenamiento. Cada input esta formado por una lista con los siguientes valores: 
* Valor de cierre para los _n_ días anteriores al entrenamiento.
* El valor de seno para el día de la semana a predecir.
* El valor del coseno para el día de la semana a predecir.

También, para la salida del entrenamiento generamos una lista de listas, donde cada sublista tiene el valor de cierre para el día a predecir.

In [4]:
training_inputs, training_outputs = dataset_manager.generateTrainingIOs(training_days, myTrainset.dias_hacia_atras)
validation_inputs, validation_outputs = dataset_manager.generateTrainingIOs(validation_days, myTrainset.dias_hacia_atras)
test_inputs, test_outputs = dataset_manager.generateTrainingIOs(test_days, myTrainset.dias_hacia_atras)

Una vez tenemos las inputs y outputs a utilizar en el modelo, creamos este seteando sus distintos parámetros.

In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, input_dim=6, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(100, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(50, activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dense(1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='mean_squared_error',
    metrics=['mae']
)

print("Resumen del modelo:")
print(model.summary())
print("Comenzando entrenamiento...")
historial = model.fit(training_inputs, training_outputs, epochs=3000 ,verbose=1, validation_data=(validation_inputs, validation_outputs))
print("Modelo entrenado!")
print("Guardando modelo...")
model.save('models/model-paper.h5', historial)

Resumen del modelo:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                350       
                                                                 
 dense_1 (Dense)             (None, 100)               5100      
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 10,551
Trainable params: 10,551
Non-trainable params: 0
_________________________________________________________________


2022-10-04 16:41:14.207598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-04 16:41:14.208104: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-04 16:41:14.208192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-04 16:41:14.208288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-04 16:41:14.208365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

None
Comenzando entrenamiento...
Epoch 1/3000
13/13 [==============================] - 1s 21ms/step - loss: 19095826.0000 - mae: 4344.6313 - val_loss: 27553038.0000 - val_mae: 5247.9238
Epoch 2/3000
13/13 [==============================] - 0s 7ms/step - loss: 18562434.0000 - mae: 4283.6543 - val_loss: 26783948.0000 - val_mae: 5174.1626
Epoch 3/3000
13/13 [==============================] - 0s 6ms/step - loss: 18042290.0000 - mae: 4223.0894 - val_loss: 26028174.0000 - val_mae: 5100.6387
Epoch 4/3000
13/13 [==============================] - 0s 6ms/step - loss: 17528334.0000 - mae: 4162.7158 - val_loss: 25289954.0000 - val_mae: 5027.7847
Epoch 5/3000
13/13 [==============================] - 0s 6ms/step - loss: 17030408.0000 - mae: 4102.9253 - val_loss: 24563866.0000 - val_mae: 4955.0835
Epoch 6/3000
13/13 [==============================] - 0s 7ms/step - loss: 16539372.0000 - mae: 4043.4021 - val_loss: 23856472.0000 - val_mae: 4883.2124
Epoch 7/3000
13/13 [==============================] - 

Cargamos el modelo guardado

In [6]:
print("Cargando modelo...")
model = tf.keras.models.load_model('models/model-paper.h5')

Cargando modelo...


Ahora realizamos predicciones sobre el conjunto de testing y creamos un Dataset con los valores reales y la fecha.

In [ ]:
predictions = [item for sublist in model.predict(test_inputs) for item in sublist]
results_df = pd.DataFrame(columns=['Date', 'Prediction', 'RealValue'])
results_df["Date"] = test_days["Date"].iloc[myTrainset.dias_hacia_atras:].apply(lambda row: (pd.to_datetime(row)).strftime("%Y-%m-%d"))
results_df["RealValue"] = test_days["Close"].iloc[myTrainset.dias_hacia_atras:]
results_df["Prediction"] = predictions


Finalmente graficamos los resultados y calculamos el R2


In [ ]:
plt.plot(results_df["Prediction"], marker='o', color='green')
plt.xlabel("Fecha")
plt.xticks(ticks=range(0,len(results_df)), labels=results_df["Date"], rotation=90)
plt.ylabel("Valor de cierre")
plt.plot(results_df["RealValue"], marker="s", color='red')

print(f'R2: {r2_score(results_df["RealValue"], results_df["Prediction"])}')

plt.show()